# Pivoter des données

On utilise le même tableau de données que dans 04-Pandas-Merge.

Ce fichier contient en fait deux tableaux l’un derrière l’autre.

Cette fois, au lieu de supprimer les données sur les décès, on va améliorer la structure du tableau.

In [10]:
import pandas as pd

In [11]:
# On charge un fichier détaillé de l’OFSP dans sa version du 3 septembre 2020

df = pd.read_csv('data/Dashboards_1&2_COVID19_swiss_data_pv-2.csv')

# En principe, on pouvait aussi charger la dernière version directement sur le site de l’OFSP.
# Ça ne marche plus (mais vous pouvez essayer, on sait jamais).
# df = pd.read_excel('https://www.bag.admin.ch/dam/bag/fr/dokumente/mt/k-und-i/aktuelle-ausbrueche-pandemien/2019-nCoV/covid-19-basisdaten-fallzahlen.xlsx.download.xlsx/Dashboards_1&2_COVID19_swiss_data_pv.xlsx')

/var/folders/fk/s7sy03296blddgxfssmkxdzw0000gn/T/ipykernel_98801/2230824978.py:3: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/Dashboards_1&2_COVID19_swiss_data_pv-2.csv')


Pandas a dû vous donner une erreur selon laquelle «des colonnes ont des types mixtes». Ça veut dire qu’il n’a pas pu identifier clairement quelle colonne contient une date, un entier, des float, etc. On va examiner ça.

In [12]:
# Combien de lignes en tout?
len(df)

304255

La colonne **replikation_dt** contient la date de mise à jour du fichier. On n’en a pas besoin. Même chose pour la colonne **Geschlecht**, qui est à double.

In [13]:
del df['replikation_dt']
del df['Geschlecht']

Les autres colonnes sont expliquées dans un fichier de l’OFSP, on va leur donner un nom compréhensible…

In [14]:
df.columns = ['date cas', 'canton', 'age', 'genre_num', 'genre', 'nouveaux cas',
       'date décès', 'nouveaux décès']

C’est mieux!

et maintenant…

## Faire un pivot

On choisit:
- ce qu'on veut comme index
- ce qu'on veut comme colonnes
- ce qu'on veut come valeurs
- comment on veut les agréger: somme, moyenne

Deux fonctions sont disponibles:
* `pd.pivot_table()`: on peut agréger les valeurs (somme, moyenne, etc.)
* `pd.pivot()`: pas de modification des valeurs


In [6]:
df.head(2)

,date cas,canton,age,genre_num,genre,nouveaux cas,date décès,nouveaux décès
0,2020-02-24,TI,70 - 79,1,homme,1,NaN,0
1,2020-02-25,AG,20 - 29,1,homme,1,NaN,0


In [18]:
# Faire la somme des cas et des décès, toute date confondue, avec pivot_table
pd.pivot_table(
    df,
    index="age",
    columns="canton",
    values=["nouveaux cas", "nouveaux décès"],
    aggfunc="sum",
)

nouveaux cas                                            ...  \
canton              AG  AI  AR   BE   BL   BS  FL   FR    GE  GL  ...   
age                                                               ...   
0 - 9               22   0   0   35   12   14   0    5    87   0  ...   
10 - 19            149   0  13  155   45   51   8  107   318  18  ...   
20 - 29            449   3  18  490  178  239  26  389  1250  39  ...   
30 - 39            324   0  10  392  150  231  23  210  1269  16  ...   
40 - 49            319   3  19  349  187  217  14  251  1299  20  ...   
50 - 59            391  13  30  477  230  251  15  242  1278  24  ...   
60 - 69            239   2  15  283  120  133  14  149   603  19  ...   
70 - 79            105   3  11  176   80  115   3  153   399   8  ...   
80+                133   1  10  220   94  133   5  264   671  22  ...   
Unbekannt            0   0   0    4    2    2   0    1     4   0  ...   

          nouveaux décès                                      
canton                SH  SO  SZ  TG   TI UR   VD  VS ZG  ZH  
age                                                           
0 - 9                  0   0   0   0    0  0    0   0  0   0  
10 - 19                0   0   0   0    0  0    0   0  0   0  
20 - 29                0   0   0   0    0  0    0   0  0   0  
30 - 39                0   0   1   0    0  0    1   0  0   0  
40 - 49                0   0   1   0    2  0    0   0  0   0  
50 - 59                1   1   1   1    6  0    2   2  0   5  
60 - 69                1   1   4   2   27  0   25   5  1  14  
70 - 79                0   4   8   4   77  4   56  21  2  22  
80+                    1  12  10  12  200  3  222  78  4  98  
Unbekannt              0   0   0   0    0  0    0   0  0   0  

[10 rows x 54 columns]

In [19]:
pd.pivot(
    df[(df["canton"] == "GE") & (df["date cas"].notnull())],
    index="date cas",
    columns=["age", "genre"],
    values="nouveaux cas",
)

age        20 - 29       30 - 39 50 - 59 30 - 39 40 - 49 10 - 19 70 - 79  \
genre        homme femme   homme   homme   femme   homme   homme   homme   
date cas                                                                   
2020-02-24       0     0       0       0       0       0       0       0   
2020-02-25       0     0       0       0       0       0       0       0   
2020-02-26       1     0       0       0       0       0       0       0   
2020-02-27       0     1       1       1       0       0       0       0   
2020-02-28       1     1       1       0       1       1       0       0   
...            ...   ...     ...     ...     ...     ...     ...     ...   
2020-08-29       7     9       4       2       6       6       0       0   
2020-08-30       6     3       4       2       3       2       0       0   
2020-08-31       9     9       7       2      10       7       1       0   
2020-09-01       8     5       7       4       6       1       1       0   
2020-09-02       5     8       5       3       7       4       0       0   

age        50 - 59 60 - 69  ...  20 - 29  30 - 39    0 - 9  10 - 19  50 - 59  \
genre        femme   femme  ... manquant manquant manquant manquant manquant   
date cas                    ...                                                
2020-02-24       0       0  ...        0        0        0        0        0   
2020-02-25       0       0  ...        0        0        0        0        0   
2020-02-26       0       0  ...        0        0        0        0        0   
2020-02-27       0       0  ...        0        0        0        0        0   
2020-02-28       0       0  ...        0        0        0        0        0   
...            ...     ...  ...      ...      ...      ...      ...      ...   
2020-08-29       2       0  ...        0        0        0        0        0   
2020-08-30       4       2  ...        0        0        0        0        0   
2020-08-31       3       0  ...        0        0        0        0        0   
2020-09-01       4       0  ...        0        0        0        0        0   
2020-09-02       4       0  ...        0        0        0        0        0   

age         60 - 69  70 - 79      80+ Unbekannt           
genre      manquant manquant manquant     homme manquant  
date cas                                                  
2020-02-24        0        0        0         0        0  
2020-02-25        0        0        0         0        0  
2020-02-26        0        0        0         0        0  
2020-02-27        0        0        0         0        0  
2020-02-28        0        0        0         0        0  
...             ...      ...      ...       ...      ...  
2020-08-29        0        0        0         0        0  
2020-08-30        0        0        0         0        0  
2020-08-31        0        0        0         0        0  
2020-09-01        0        0        0         0        0  
2020-09-02        0        0        0         0        0  

[192 rows x 30 columns]

In [23]:
# Additionner hommes et fammes
pd.pivot_table(
    df[(df["canton"] == "GE") & (df["date cas"].notnull())],
    index="date cas",
    columns="age",
    values="nouveaux cas",
    aggfunc="sum",
)

age,0 - 9,10 - 19,20 - 29,30 - 39,40 - 49,50 - 59,60 - 69,70 - 79,80+,Unbekannt
date cas,,,,,,,,,,
2020-02-24,0,0,0,0,0,0,0,0,0,0
2020-02-25,0,0,0,0,0,0,0,0,0,0
2020-02-26,0,0,1,0,0,0,0,0,0,0
2020-02-27,0,0,1,1,0,1,0,0,0,0
2020-02-28,0,0,2,2,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2020-08-29,3,2,16,10,9,4,1,0,1,0
2020-08-30,5,2,9,7,4,6,2,0,0,0
2020-08-31,5,3,18,17,9,5,0,0,2,0
